In [1]:
from selenium import webdriver

# ブラウザの起動
driver = webdriver.Chrome()
# URLを指定してWebページを開く
driver.get("https://example.com")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from datetime import datetime
import re
import os

DATA_DIR = "../data"

def validate_date_format(date_str: str) -> bool:
  """
  日付文字列がYYYY-MM-DD形式かどうかを検証する
  
  Args:
      date_str (str): 検証する日付文字列
      
  Returns:
      bool: 有効な形式の場合はTrue、そうでない場合はFalse
  """
  pattern = r'^\d{4}-\d{2}-\d{2}$'
  if not re.match(pattern, date_str):
    return False
  
  try:
    datetime.strptime(date_str, '%Y-%m-%d')
    return True
  except ValueError:
    return False

def get_stock_csv(code: str, from_date: str, to_date: str):
  """
  株価データを取得し、CSVファイルに保存する

  Args:
      code (str): 株価データを取得する銘柄コード
      from_date (str): 株価データを取得する開始日
      to_date (str): 株価データを取得する終了日

  Raises:
      ValueError: 日付の形式が不正です。YYYY-MM-DD形式で入力してください。
  """
  if not validate_date_format(from_date) or not validate_date_format(to_date):
    raise ValueError("日付の形式が不正です。YYYY-MM-DD形式で入力してください。")
  
  if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
  
  from_date_epoch = int(datetime.strptime("{}T09:00:00".format(from_date), "%Y-%m-%dT%H:%M:%S").timestamp())
  to_date_epoch = int(datetime.strptime("{}T00:00:00".format(to_date), "%Y-%m-%dT%H:%M:%S").timestamp())
  
  url = "https://finance.yahoo.com/quote/{}/history/?period1={}&period2={}".format(code, from_date_epoch, to_date_epoch)
  print(f"アクセスURL: {url}")
  
  # Chromeオプションを設定（ヘッドレスモード）
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument("--no-sandbox")
  
  # WebDriverを初期化
  with webdriver.Chrome(options=chrome_options) as driver:
  
    # ページにアクセス
    driver.get(url)
    
    # テーブルデータを取得
    try:
      
      #"table.hideOnPrint"のテーブルをawaitで取得
      table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table.hideOnPrint"))
      )
      rows = table.find_elements(By.TAG_NAME, "tr")
      
      file_name = "{}/stock_data_{}_{}_{}.csv".format(DATA_DIR, code, from_date, to_date)
      
      with open(file_name, "w", encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Code", "Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])
        
        for row in rows[1:]:  # ヘッダー行をスキップ
          cells = row.find_elements(By.TAG_NAME, "td")
          code_arr = [code]
          if cells:
            row_data = code_arr + [cell.text for cell in cells]
            if any(row_data):  # 空行でない場合のみ書き込み
              writer.writerow(row_data)
      
      print(f"CSVファイルを保存しました: {file_name}")
      
    except Exception as e:
      print(f"テーブルデータの取得に失敗: {e}")
    
    driver.quit()

In [2]:
fromDate = "2024-04-01"
toDate = "2025-04-18"
code = "AMZN"

get_stock_csv(code, fromDate, toDate)

アクセスURL: https://finance.yahoo.com/quote/AMZN/history/?period1=1711929600&period2=1744902000
CSVファイルを保存しました: ../data/stock_data_AMZN_2024-04-01_2025-04-18.csv
